In [1]:
import sys
sys.path.append('../')

import os
import json
import pandas as pd

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
data = []
with open(os.path.join('../data/json/challenges/Ferrara2023-citizens-individualChallenges.json')) as f:
    data = [json.loads(line) for line in f]

display(len(data))
data = [challenge for challenge in data if challenge['concept']['modelName'] not in ['survey']]
display(len(data))
data = [pd.json_normalize(d, sep='.') for d in tqdm(data)]


7172

6647

  0%|          | 0/6647 [00:00<?, ?it/s]

In [3]:
df_individual = pd.concat(data, ignore_index=True)
df_individual['start'] = pd.to_datetime(df_individual['concept.start'], cache=True)
df_individual['week'] = df_individual['start'].dt.week
df_individual['year'] = df_individual['start'].dt.year
df_individual['target'] = df_individual['concept.fields.target'].astype(int)
df_individual = df_individual[[
    'playerId',
    'start',
    'week',
    'year',
    'concept.modelName',
    'concept.fields.counterName',
    'target',
    'concept.fields.periodTarget',
    'concept.state']]
df_individual.rename(columns={
    'concept.fields.counterName': 'counterName', 
    'concept.fields.periodTarget': 'periodTarget', 
    'concept.state': 'state',
    'concept.modelName': 'modelName'}, inplace=True)


In [14]:
df_individual.head()

,playerId,start,week,year,modelName,counterName,target,periodTarget,state
0,u_0bea6988-bd00-4aa6-a456-4285744356ee,2023-04-23 22:00:00,16,2023,absoluteIncrement,Walk_Km,1,None,COMPLETED
1,u_0bea6988-bd00-4aa6-a456-4285744356ee,2023-04-23 22:00:00,16,2023,repetitiveBehaviour,green leaves,30,2,COMPLETED
2,u_1636dfdc-fbcc-4068-8fcd-3293369c3a82,2023-04-23 22:00:00,16,2023,absoluteIncrement,Walk_Km,1,None,COMPLETED
3,u_1636dfdc-fbcc-4068-8fcd-3293369c3a82,2023-04-23 22:00:00,16,2023,repetitiveBehaviour,green leaves,30,2,COMPLETED
4,u_2fe7aac8-07da-4d38-8b0a-978be1986ebf,2023-04-23 22:00:00,16,2023,absoluteIncrement,Walk_Km,1,None,COMPLETED


In [5]:
data = []
with open(os.path.join('../data/json/challenges/Ferrara2023_citizens_groupChallenges.json')) as f:
    data = [json.loads(line) for line in f]

In [6]:
df_group = pd.DataFrame(data)
df_group['counterName'] = df_group['challengePointConcept'].apply(lambda x: x['name'])
df_group['start'] = pd.to_datetime(df_group['start'], cache=True)
df_group['week'] = df_group['start'].dt.week
df_group['year'] = df_group['start'].dt.year
df_group['modelName'] = 'absoluteIncrement'
df_group['periodTarget'] = None
df_group = df_group.explode('attendees')
df_group['playerId'] = df_group['attendees'].apply(lambda x: x['playerId'])
df_group['target'] = df_group['attendees'].apply(lambda x: x['challengeScore']).astype(int)

def compute_state(x):
    if x['state'] == 'FAILED' or x['state'] == 'ASSIGNED':
        return x['state']
    if str(x['attendees']['isWinner']) == 'True':
        return 'COMPLETED'
    return 'FAILED'

df_group['state'] = df_group.apply(compute_state, axis=1)

df_group = df_group[[
    'playerId', 
    'start', 
    'week', 
    'year', 
    'modelName', 
    'counterName', 
    'target',
    'periodTarget',
    'state']]
df_group.head(20)

,playerId,start,week,year,modelName,counterName,target,periodTarget,state
0,u_1636dfdc-fbcc-4068-8fcd-3293369c3a82,2023-04-30 22:00:00,17,2023,absoluteIncrement,Bike_Km,99,None,COMPLETED
0,u_637eccb3-b595-4e1d-b387-ad292051b5b9,2023-04-30 22:00:00,17,2023,absoluteIncrement,Bike_Km,30,None,COMPLETED
1,u_0bea6988-bd00-4aa6-a456-4285744356ee,2023-04-30 22:00:00,17,2023,absoluteIncrement,Walk_Km,15,None,COMPLETED
1,u_f2d8d95d-6cc9-4e8e-b97d-88d0c1b0ae27,2023-04-30 22:00:00,17,2023,absoluteIncrement,Walk_Km,5,None,COMPLETED
2,u_2fe7aac8-07da-4d38-8b0a-978be1986ebf,2023-04-30 22:00:00,17,2023,absoluteIncrement,green leaves,158,None,COMPLETED
2,u_c790f7da93484fc484eb7bdd0303fa47,2023-04-30 22:00:00,17,2023,absoluteIncrement,green leaves,372,None,COMPLETED
3,u_7d3ebab3-b62a-4df9-bf3e-5655f6d37b8e,2023-04-30 22:00:00,17,2023,absoluteIncrement,green leaves,261,None,COMPLETED
3,u_a1bb70d75a3f442ba32cd34ad034a25a,2023-04-30 22:00:00,17,2023,absoluteIncrement,green leaves,459,None,COMPLETED
4,u_489d0f12-79e5-4207-b5f9-1e8394175594,2023-04-30 22:00:00,17,2023,absoluteIncrement,green leaves,360,None,COMPLETED
4,u_4336c3483c88493cb1d734c13c0dc372,2023-04-30 22:00:00,17,2023,absoluteIncrement,green leaves,370,None,COMPLETED
